In [3]:
import requests
from binance.client import Client
import datetime
import pandas as pd
import json

def get_crypto_hystorical_data(symbol, days_in_the_past):
    client = Client()
    untilThisDate = datetime.datetime.now()
    sinceThisDate = untilThisDate - datetime.timedelta(days = days_in_the_past)
    
    # Execute the query from binance - timestamps must be converted to strings !
    candle = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_30MINUTE, str(sinceThisDate), str(untilThisDate))

    # Create a dataframe to label all the columns returned by binance so we work with them later.
    df = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])

    # Get rid of columns we do not need
    df = df.drop(['closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol','takerBuyQuoteVol', 'ignore'], axis=1)

    return df

In [85]:
import numpy as np
import tensorflow as tf
from joblib import load as joblib_load

def get_crypto_hystorical_data_for_prediction():
    def create_series(data, lookback=7*48):  # 7 days, 48 intervals per day
        if len(data) >= lookback:
            return data[-lookback:]  # Return a 2D array
        else:
            print("Not enough data points. Need at least {} data points.".format(lookback))
            return []

    # get the last 7 days of crypto data
    crypto_history_df = get_crypto_hystorical_data('BTCUSDT', 9)

    # Load the scaler
    scaler = joblib_load("../models/scaler.pkl")

    # Normalize the 'close' column
    crypto_history_data = scaler.transform(crypto_history_df[['close']])

    # Create 7-day series
    crypto_history_data = create_series(crypto_history_data)

    return crypto_history_data


In [87]:
crypto_history_data = get_crypto_hystorical_data_for_prediction()

crypto_history_data.shape
crypto_history_data

array([[0.91498247],
       [0.91104424],
       [0.9062231 ],
       [0.9002237 ],
       [0.90874058],
       [0.91376864],
       [0.91018413],
       [0.91722776],
       [0.91522773],
       [0.9146747 ],
       [0.9098349 ],
       [0.90883454],
       [0.91035339],
       [0.91028465],
       [0.91139002],
       [0.90724488],
       [0.9028227 ],
       [0.90208487],
       [0.88803699],
       [0.89640602],
       [0.89948725],
       [0.90095808],
       [0.89501671],
       [0.89362533],
       [0.89446023],
       [0.88496405],
       [0.86525909],
       [0.88603557],
       [0.88071632],
       [0.88773163],
       [0.87263295],
       [0.88582797],
       [0.89551413],
       [0.89453277],
       [0.8873862 ],
       [0.89802159],
       [0.92506518],
       [0.92441198],
       [0.92939824],
       [0.92076736],
       [0.92808043],
       [0.90263514],
       [0.9155697 ],
       [0.90376469],
       [0.90931745],
       [0.92515672],
       [0.94236424],
       [0.939

In [93]:
def predict_crypto():

    crypto_history_data = get_crypto_hystorical_data_for_prediction()
    
    # load the model from disk
    model = tf.keras.models.load_model("../models/crypto_prediction.h5")
    
    # Assuming 'model' is your trained LSTM model
    predictions = []
    
    for _ in range(336):  # For each 30-minute interval of the last week
        # Use the last 336 instances to predict the next instance
        X_test = crypto_history_data[-336:]
        X_test = np.reshape(X_test, (1, X_test.shape[0], 1))  # Reshape to be 3-dimensional
        pred = model.predict(X_test)
        
        # Append the predictions to the 'predictions' list
        predictions.extend(pred[0])
        
        # Slide the window forward by 48 instances
        crypto_history_data = np.concatenate((crypto_history_data, pred))
    
    return predictions

In [94]:
predictions = predict_crypto()

print(predictions)

1/1 [==============================] - 0s 116ms/step
[0.81487393, 0.8121217, 0.8098492, 0.80813175, 0.80689335, 0.80601925, 0.80540305, 0.80496126, 0.804631, 0.80436766, 0.8041383, 0.8039201, 0.8036975, 0.8034608, 0.80320424, 0.8029257, 0.80262524, 0.8023042, 0.80196524, 0.8016109, 0.80124426, 0.800868, 0.8004844, 0.80009586, 0.79970396, 0.79931015, 0.7989158, 0.7985216, 0.7981282, 0.79773617, 0.79734576, 0.7969573, 0.7965709, 0.7961865, 0.79580444, 0.79542416, 0.79504603, 0.79466987, 0.79429555, 0.7939231, 0.7935524, 0.79318345, 0.792816, 0.7924502, 0.7920859, 0.79172283, 0.7913611, 0.79100096, 0.79064184, 0.79028416, 0.7899275, 0.7895722, 0.78921795, 0.7888651, 0.7885132, 0.78816265, 0.78781307, 0.78746486, 0.7871175, 0.7867714, 0.7864266, 0.7860828, 0.78574, 0.7853986, 0.7850581, 0.78471875, 0.7843806, 0.78404355, 0.78370756, 0.7833728, 0.78303903, 0.7827065, 0.7823749, 0.7820445, 0.78171504, 0.7813869, 0.7810596, 0.7807334, 0.78040826, 0.7800844, 0.7797614, 0.7794394, 0.77911824, 0